# Probleme :
Le postagger ne gère pas des données trop volumineuse. Soit réglage du postagger/ paramètre serveur, soit fractionner la donnée et répartir en plusieurs execution.

In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import re
#from nltk.tag.stanford import StanfordPOSTagger
from nltk.stem import SnowballStemmer

# Récupération des documents sous forme de dataset

In [2]:
######################
# Read recipe inputs #
######################
etudes_dataset = dataiku.Dataset("etudes_dataset_EXP_joined")
etudes_dataset_df = etudes_dataset.get_dataframe()

# Fonction de pré-traitement avec sélection des features

In [3]:
path = '../../../../uploads/ETUDE_IMPACT/datasets/postagger/'
#pos_tagger = StanfordPOSTagger(path+"french-ud.tagger", path+"stanford-postagger.jar", encoding='utf8')

frenchStem = SnowballStemmer('french')

In [4]:
def pretraitement(chaine, tolower=True, deletePunctuation=True, replaceAccent=True, deleteSpecialChar=True, stopwords=None, postagging=None, stemming=True):
    """
    Fonction permettant le prétraitement des chaines de caractères.

    Le remplacement des accents permets de ne pas avoir de problème liés à une orthographe différentes de deux mots au niveau des accents.
    La suppression des caractères spéciaux supprime tous les caractères sauf ceux alphabétique, la ponctuation et le tiret.

    :param chaine: La chaine de caractère à traiter
    :param tolower: booleen permettant la mise en minuscule
    :param deletePunctuation: booleen permettant la suppression des ponctuations
    :param replaceAccent: booleen permettant le remplacement des caractères accentués par leur version non accentuée
    :param deleteSpecialChar: booleen permettant la suppression des caratères spéciaux (tous les caractères non alphabetiques)
    :param stopwords: Liste de stopwords à supprimer
    :param postagging: Liste de type de mots à garder (Les types possibles sont : PRON, DET, VERB, NOUN, ADJ, CCONJ, PROPN, ADV)

    :type chaine: string
    :type tolower: boolean
    :type deletePunctuation: boolean
    :type replaceAccent: boolean
    :type deleteSpecialChar: boolean
    :type stopwords: list[string]
    :param postagging: list[string]

    :return: La chaine prétraitée
    :rtype: string
    """
    res = ""

    if(tolower==True):
        res = chaine.lower()

    if(deletePunctuation==True):
        res = re.sub('[,;?:.!]', ' ', res)

    if(replaceAccent==True):
        res = res.replace('â','a').replace('à','a').replace('ê','e').replace('é','e').replace('è','e').replace('ë','e')
        res = res.replace('û','u').replace('ù','u').replace('î','i').replace('ï','i').replace('ç','c')

    if(deleteSpecialChar==True):
        res = re.sub('[^a-z ,;:.?!-]', ' ', res)


    res = res.split(" ")

    if(stopwords != None):
        res = [word for word in res if len(word)>0 and word not in stopwords]
    else:
        res = [word for word in res if len(word)>0]

    #postagging
    if(postagging!=None):
        resultat = res
        #lim = 10000
        #resultat = []
        #for i in range(int(np.ceil(len(res)/lim))):
        #    print("OK")
        #    for word, tag in pos_tagger.tag(res[i:(i+1)*lim]):
        #        if(tag in postagging and word not in resultat):
        #            resultat.append(word)
    else:
        resultat = res


    #stemming
    if(stemming==True):
        resultat = [frenchStem.stem(word) for word in resultat]

    return " ".join(resultat)

# Application des traitements aux documents

In [5]:
# Application des pré-traitements choisis à chaque document (1 par ligne)
docs = []
for index, line in etudes_dataset_df.iterrows():
    docs.append(pretraitement(line['docs'], postagging=None, stemming=False))
    
etudes_dataset_cleaned_df = etudes_dataset_df.copy(deep=True)
etudes_dataset_cleaned_df['docs'] = pd.Series(docs)

# Ecriture des données dans un dataset Dataiku

In [0]:
########################
# Write recipe outputs #
########################
etudes_dataset_cleaned = dataiku.Dataset("etudes_dataset_cleaned_EXP")
etudes_dataset_cleaned.write_with_schema(etudes_dataset_cleaned_df)